<a href="https://colab.research.google.com/github/luisosmx/API_connections/blob/main/API_ACROBAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

url = "https://secure.echosign.com/public/oauth/v2"
redirect_uri = "https://oauth.pstmn.io/v1/callback"
client_id = ""
scopes = [
    "user_read:account",
    "user_write:account",
    "user_login:account",
    "agreement_read:account",
    "agreement_write:account",
    "agreement_send:account",
    "library_read:account",
    "library_write:account"
]

parameters = {
    "redirect_uri": redirect_uri,
    "response_type": "code",
    "client_id": client_id,
    "scope": " ".join(scopes)
}

response = requests.get(url, params=parameters)

print("Status code:", response.status_code)
print("Response URL:", response.url)





Status code: 200
Response URL: https://secure.na1.echosign.com/public/oauth/v2?redirect_uri=https%3A%2F%2Foauth.pstmn.io%2Fv1%2Fcallback&response_type=code&client_id=CBJCHBCAABAA5z5y6_YYaecwliRsbOJke0uGa4pet5Tr&scope=user_read%3Aaccount+user_write%3Aaccount+user_login%3Aaccount+agreement_read%3Aaccount+agreement_write%3Aaccount+agreement_send%3Aaccount+library_read%3Aaccount+library_write%3Aaccount


In [ ]:
import requests

# Configura las credenciales de autenticación
client_id = 'CBJCHBCAABAAnCmyw0Ko9POVpVCxMJnDPM_hvG9O0vB5'
client_secret = 'LgLmlC83KXAufmGajmw22Keh10kMUMbm'
access_token = None

# Obtiene un token de acceso usando las credenciales de OAuth
def get_access_token():
    global access_token
    url = 'https://ims-na1.adobelogin.com/ims/token/v1'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    }
    response = requests.post(url, headers=headers, data=data)
    response_json = response.json()
    print(response_json)
    access_token = response_json['access_token']

# Función para cargar un documento
def upload_document(file_path):
    if access_token is None:
        get_access_token()
    url = 'https://api.acrobat.com/apiservices/v4/upload'
    headers = {
        'Authorization': f'Bearer {access_token}',
    }
    files = {'file': open(file_path, 'rb')}
    response = requests.post(url, headers=headers, files=files)
    response_json = response.json()
    return response_json['id']

# Función para enviar un documento
def send_document(document_id, recipient_email):
    if access_token is None:
        get_access_token()
    url = 'https://api.acrobat.com/apiservices/v4/sendAgreement'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }
    data = {
        'documentId': document_id,
        'participantSetsInfo': [
            {
                'memberInfos': [
                    {
                        'email': recipient_email,
                    }
                ],
                'order': 1,
            }
        ]
    }
    response = requests.post(url, headers=headers, json=data)
    response_json = response.json()
    return response_json

# Ejemplo de uso
if __name__ == '__main__':
    document_id = upload_document('/content/Guia-Corredor-DEN24.pdf')
    recipient_email = 'luis.osmx16@gmail.com'
    response = send_document(document_id, recipient_email)
    print(response)


{'error': 'invalid_client', 'error_description': 'invalid client_id parameter'}


KeyError: 'access_token'